In [1]:
import os
from os import path
import sqlite3
global_sqlite_filename='apf.db'
import pandas
from datetime import datetime, timedelta
import numpy as np
from astropy.time import Time
from sqlalchemy import create_engine
import pytz
import numpy as np

In [2]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

#print(os.getcwd())
#print(os.listdir())

TEXT


In [3]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def get_velocity_columns():
    columns = [('MdptJulian', ct.REAL),
               ('velocity', ct.REAL),
               ('velocity_error', ct.REAL),
               ('stellar_activity1', ct.REAL),
               ('stellar_activity2', ct.REAL),
               ('total_counts', ct.REAL),
               ('brad_doesnt_know', ct.REAL),
               ('DateTimeUTC', ct.TEXT),
               ('DateTimeLocal', ct.TEXT),
               ('starID', ct.TEXT)
              ]
    return columns
    
def create_telemetry_table(cursor):        
    columns = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
    cursor.execute(sql_create_string)

    return


def create_velocity_table(cursor):
    columns = get_velocity_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns]
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE velocity (' + full_list + ', PRIMARY KEY (DateTimeLocal))'
    print(sql_create_string)  
    cursor.execute(sql_create_string)

    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [4]:
conn = open_connection()
cursor = conn.cursor()
#create_telemetry_table(cursor)
#create_velocity_table(cursor)

In [5]:
def populate_telemetry_table(conn, filenames):
    for csv_file in filenames:
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        
    


        

In [6]:
#populate_telemetry(conn, '../telemetry_data', batchsize=10)

In [7]:
def load_velocity_csv(dirname, filename):
    full_path = os.path.join(dirname, filename)
    columns = get_velocity_columns()[:-2] # leave off the last two column names
                                        # because we're calculating them below.
    df = pandas.read_csv(full_path, sep='\s+',
            names=[x[0] for x in columns]) 
                                                
    return df
    
    
california_tzinfo = pytz.timezone('US/Pacific')

# returns a string for a single julian date. Couldn't figure out how to vectorize.
def convert_julian_to_datetime_local(juliantime):
    times = Time(juliantime, format='jd')
    full_iso = times.to_datetime(timezone=california_tzinfo).isoformat()
    return full_iso[:-6]

# takes in an array of julian dates. Returns an array of local isot strings.
def make_array_of_local_datetimes(julian_array):
    output_array = output_array = np.chararray(len(julian_array), itemsize=26, unicode=True)
    for i, julian_time in enumerate(julian_array):
        output_array[i] = convert_julian_to_datetime_local(julian_time)
    return output_array
    

def convert_julian_to_datetime_utc(juliantimes):
    times = Time(juliantimes, format='jd')
    return times.isot 
    
def add_calculated_columns(df, filename):
    new_df = df.copy()
    #datetimes = convert_julian_to_datetime(new_df.MdptJulian)
    new_df['DateTimeUTC'] = convert_julian_to_datetime_utc(new_df.MdptJulian)
    new_df['DateTimeLocal'] = make_array_of_local_datetimes(new_df.MdptJulian)
    new_df['starID'] = [filename] * len(new_df.index) 
    
    return new_df
    

def populate_velocity(conn, dirname='../standard_star_velocity_measurements/', debug=False):
    filenames = ['HD10700_APF.vels', 'HD185144_APF.vels', 'HD9407_APF.vels']
    for file in filenames:
        df_incomplete = load_velocity_csv(dirname, file)
        df_complete = add_calculated_columns(df_incomplete, file)
        if debug:
            print(df_complete)
        else: 
            df_complete.to_sql('velocity', conn, if_exists='append', index=False)
        print('tried to add {} entries to \'velocity\''.format(len(df_complete)))

#velocity_df = load_velocity_csv('../standard_star_velocity_measurements/', 'HD10700_APF.vels')
#add_calculated_columns(velocity_df, 'HD10700_APF.vels' )


In [8]:
# create additional calculated columns on velocity
engine = create_engine('sqlite:///apf.db', echo=False)


In [59]:
#add new table telemetrytimeconvert
def drop_telemetry_time_convert(engine):
    engine.execute("""DROP TABLE IF EXISTS telemetry_time_convert;""")
    print('dropped telemetry_time_convert.')

def create_telemetry_time_convert(engine):
    engine.execute("""
    CREATE TABLE telemetry_time_convert (
    DateTime TEXT PRIMARY KEY,
    DateTimeStrfTime REAL,
    Event TEXT,
    ExposureID TEXT,
    FOREIGN KEY (DateTime) REFERENCES telemetry(DateTime) 
    )""")
    print('created telemetry_time_convert.')

def populate_telemetry_time_convert(engine):   
    engine.execute("""INSERT INTO telemetry_time_convert (DateTime,
        DateTimeStrfTime, Event) 
    SELECT t.DateTime, strftime('%s', t.DateTime) + strftime('%f', t.DateTime), t.Event
    from telemetry t 
    """)
    print('populated created telemetry_time_convert.')
    
    
#drop_telemetry_time_convert(engine)
#create_telemetry_time_convert(engine)
#populate_telemetry_time_convert(engine)
df = pandas.read_sql_query("""select * from telemetry_time_convert limit 10""", con=engine)
pandas.set_option('precision', 18)
print(df)



dropped telemetry_time_convert.
created telemetry_time_convert.
populated created telemetry_time_convert.
                   DateTime          DateTimeStrfTime            Event  \
0  2015-01-01T17:00:00.0000  1.420131600000000000e+09  ControllerReady   
1  2015-01-01T17:00:01.0570  1.420131602056999922e+09  ControllerReady   
2  2015-01-01T17:00:02.1137  1.420131604114000082e+09  ControllerReady   
3  2015-01-01T17:00:04.0934  1.420131608092999935e+09  ControllerReady   
4  2015-01-01T17:00:05.1432  1.420131610142999887e+09  ControllerReady   
5  2015-01-01T17:00:07.0867  1.420131614086999893e+09  ControllerReady   
6  2015-01-01T17:00:08.0872  1.420131616086999893e+09  ControllerReady   
7  2015-01-01T17:00:09.9234  1.420131618923000097e+09  ControllerReady   
8  2015-01-01T17:00:11.0724  1.420131622072000027e+09  ControllerReady   
9  2015-01-01T17:00:12.0986  1.420131624098999977e+09  ControllerReady   

  ExposureID  
0       None  
1       None  
2       None  
3       None  
4   

In [32]:
print(pandas.read_sql_query("""select tt.DateTime, 
                            t.Event,
                            tt.DateTimestrftime,
                            tt.DateTimeJulian
                            FROM telemetry_time_convert tt 
                            INNER JOIN telemetry t on tt.DateTime = t.Datetime
                            where tt.Datetime between "2015-04-02T01:30:25.832007"
                            and "2015-04-03T01:32:44.832007"
                            limit 30
                        """, engine))

pandas.set_option('precision', 15)



                    DateTime            EVENT  DateTimeStrfTime  \
0   2015-04-02T01:30:26.0495  ControllerReady      1427938226.0   
1   2015-04-02T01:30:27.6908  ControllerReady      1427938227.0   
2   2015-04-02T01:30:29.0345  ControllerReady      1427938229.0   
3   2015-04-02T01:30:30.0353  ControllerReady      1427938230.0   
4   2015-04-02T01:30:31.4546  ControllerReady      1427938231.0   
5   2015-04-02T01:30:32.6018  ControllerReady      1427938232.0   
6   2015-04-02T01:30:34.0320  ControllerReady      1427938234.0   
7   2015-04-02T01:30:35.0428  ControllerReady      1427938235.0   
8   2015-04-02T01:30:37.0157  ControllerReady      1427938237.0   
9   2015-04-02T01:30:38.0273  ControllerReady      1427938238.0   
10  2015-04-02T01:30:39.0280  ControllerReady      1427938239.0   
11  2015-04-02T01:30:40.0284  ControllerReady      1427938240.0   
12  2015-04-02T01:30:41.0290       EraseBegin      1427938241.0   
13  2015-04-02T01:30:42.0367       EraseBegin      1427938242.

In [ ]:
df = pandas.read_sql_query("""select DateTimeLocal from velocity limit 10
                        """, engine)
print(df)

                DateTimeLocal
0  2013-06-28T00:20:02.111994
1  2013-06-28T00:21:49.248005
2  2013-06-28T00:23:36.383976
3  2013-06-28T03:45:06.336021
4  2013-07-05T22:47:05.856020
5  2013-07-05T22:48:59.904003
6  2013-07-05T22:50:54.816033
7  2013-07-05T22:52:49.728023
8  2013-07-05T22:54:44.640013
9  2013-07-05T22:56:39.552003


In [ ]:
#df = pandas.read_sql_query("""select distinct event from telemetry
                        
#                        limit 100
#                        """, engine)

df

,DateTimeLocal
0,2013-06-28T00:20:02.111994
1,2013-06-28T00:21:49.248005
2,2013-06-28T00:23:36.383976
3,2013-06-28T03:45:06.336021
4,2013-07-05T22:47:05.856020
5,2013-07-05T22:48:59.904003
6,2013-07-05T22:50:54.816033
7,2013-07-05T22:52:49.728023
8,2013-07-05T22:54:44.640013
9,2013-07-05T22:56:39.552003


In [ ]:
df = pandas.read_sql_query("""select distinct event from 
                        (select event from telemetry where 
                        DateTime between "2015-01-01T17:00:00.0000" and 
                        "2015-01-02T19:01:00.0000") 
                   
                        
                        """, engine)

df

,event
0,ControllerReady
1,EraseBegin
2,ExposureBegin
3,ReadoutBegin
4,ReadoutEnd
5,ExposureEnd


In [ ]:
df = pandas.read_sql_query(
"""SELECT
  event,
  COUNT(*) AS `num`
FROM
  (select * from telemetry where 
                        DateTime between "2015-02-02T17:00:00.0000" and 
                        "2015-02-03T05:01:00.0000") 
GROUP BY
  event                  
                        """, engine)

df


,EVENT,num
0,ControllerReady,20647
1,EraseBegin,43
2,ExposureBegin,10290
3,ExposureEnd,10
4,ReadoutBegin,812
5,ReadoutEnd,11


In [ ]:
df = pandas.read_sql_query(
"""SELECT DateTime, event from (select * from telemetry where DateTime < "2015-01-02T19:01:00.0000"
    and (event == "ExposureEnd" or event == 'ReadoutEnd')) order by datetime
limit 100
                        """, engine)

df


,DateTime,EVENT
0,2015-01-02T17:54:02.1588,ReadoutEnd
1,2015-01-02T17:57:25.1011,ExposureEnd
2,2015-01-02T18:01:58.2331,ExposureEnd
3,2015-01-02T18:05:05.9387,ExposureEnd
4,2015-01-02T18:05:47.2942,ReadoutEnd
5,2015-01-02T18:09:41.7525,ExposureEnd
6,2015-01-02T18:19:14.7921,ReadoutEnd
7,2015-01-02T18:21:27.1005,ReadoutEnd
8,2015-01-02T18:40:34.7239,ExposureEnd
9,2015-01-02T18:41:16.0776,ReadoutEnd


In [34]:
#df1 = pandas.read_sql_query("""
#    Select Datetimelocal, Mdptjulian from velocity 
#    where DateTimeLocal > "2015-04-01T17:01:00.0000"
#    and DateTimeLocal < "2015-04-10T17:01:00.0000"
#    order by DateTimeLocal limit 100
#
#    """, engine)
#df2 = pandas.read_sql_query(
#"""select datetime from telemetry 
#    order by (abs(strftime('%s',datetime) - abs(strftime('%s',"2015-04-02T01:31:44.832007"))))  limit 1
#                        """, engine)

#print(df1)

print(df2)

#print(pandas.read_sql_query("""
#    Select t.DateTime, v.Datetimelocal, v.Mdptjulian from (velocity v, telemetry t)
#    where v.DateTimeLocal > "2015-04-01T17:01:00.0000"
#    and v.DateTimeLocal < "2015-04-10T17:01:00.0000"
#    and (t.DateTime - v.DateTi 
#    order by DateTimeLocal limit 100
#
#    """, engine))

                   DateTime
0  2015-04-02T01:31:43.5074


0.0

In [42]:
print(pandas.read_sql_query("""select MdptJulian, DateTimeLocal, julianday(DateTimeLocal), 
                        julianday(DateTimeUTC), strftime("%s", DateTimeLocal) from velocity where datetimelocal between 
                        "2015-04-01T01:31:43.5074" and "2015-04-03T01:31:43.5074"
                        """, engine))

              MdptJulian               DateTimeLocal  \
0  2.457114855380000e+06  2015-04-02T01:31:44.832007   
1  2.457114857330000e+06  2015-04-02T01:34:33.312002   
2  2.457114859250000e+06  2015-04-02T01:37:19.200018   
3  2.457114861040000e+06  2015-04-02T01:39:53.856029   

   julianday(DateTimeLocal)  julianday(DateTimeUTC)  \
0     2.457114563713333e+06   2.457114855380000e+06   
1     2.457114565663334e+06   2.457114857330000e+06   
2     2.457114567583333e+06   2.457114859250000e+06   
3     2.457114569373333e+06   2.457114861040000e+06   

  strftime("%s", DateTimeLocal)  
0                    1427938304  
1                    1427938473  
2                    1427938639  
3                    1427938793  
